# Wikidata Bots

> Welcome!
> 
> Please download this notebook and load it into your [PAWS instance](https://hub-paws.wmcloud.org/)

This notebook provides an overview of working with Wikidata via bots using Python frameworks.

Check out the [Wikidata:Bots documentation page](https://www.wikidata.org/wiki/Wikidata:Bots) and [Wikidata:Creating a bot](https://www.wikidata.org/wiki/Wikidata:Creating_a_bot) for a full overview of the process.

## Introduction

Source: [Wikidata:Bots](https://www.wikidata.org/wiki/Wikidata:Bots)

Wikidata is meant to be edited by both humans and machines, so learning how to use bots is an invaluable step in improving your ability to have an impact on data ingestion and quality. Bots can add [interwiki links](https://www.wikidata.org/wiki/Help:Sitelinks), [labels](https://www.wikidata.org/wiki/Help:Label), [descriptions](https://www.wikidata.org/wiki/Help:Description), [statements](https://www.wikidata.org/wiki/Help:Statements), [sources](https://www.wikidata.org/wiki/Help:Sources), and can even create items, among other things.

**Note**: Wikidata has a strict bot policy to assure that they are used in a way that is beneficial to the community and project as a whole.

> In the case of any damage caused by a bot, the bot operator is asked to stop the bot. Depending on the scale of the damage, an administrator may block the bot. The bot operator is responsible for cleaning up any damage caused by the bot.

A summary of this policy and suggestions are:

- Please [create a separate account](https://www.wikidata.org/w/index.php?title=Special:CreateAccount&returnto=Wikidata%3ABots) for your bot activity
  - Name your bot after your normal username and add in the word `bot`
  - The above helps identify bot activity
    - Bot flags in our Data Lake are based on usernames and activity
  - Create a talk page with the [bot template](https://www.wikidata.org/wiki/Template:Bot)
    - Just put `{{Bot|USER_NAME}}` in the user page text
- Add an [assertion](https://www.mediawiki.org/wiki/API:Assert) to your activity to make sure the bot is logged in
  - The Python packages below will do this for you
- Add the [bots page on Wikidata](https://www.wikidata.org/wiki/Wikidata:Bots) to your watched pages as discussions related to bots happen there
- Bots are expected to respect [maxlag](https://www.mediawiki.org/wiki/Manual:Maxlag_parameter) and follow the [API etiquette](https://www.mediawiki.org/wiki/API:Etiquette)
- Request permission for a bot flag at [Wikidata:Requests_for_permissions/Bot](https://www.wikidata.org/wiki/Wikidata:Requests_for_permissions/Bot)
  - You're expected to run a test of 50-250 edits, but start the permission process before this
  - You should outline the purpose of the bot, and if it expands too much, consider making a new request
  - If accepted, an admin/bureaucrat will close the request and a bureaucrat will add the flag
- No permission is needed for the following pages
  - The operator/bot's userspace
  - The [Wikidata sandbox](https://www.wikidata.org/wiki/Wikidata:Sandbox)
  - The [Wikidata item sandbox](https://www.wikidata.org/wiki/Q4115189) (what we'll edit!)

There are also admin bots! In addition to requesting permission for the bot, a request for [admin permissions](https://www.wikidata.org/wiki/Wikidata:Requests_for_permissions/Administrator) is also necessary.

See [bot requirements](https://www.wikidata.org/wiki/Wikidata:Bots#Bot_requirements) for a full list of what is expected once your bot is up and running.

## Creating a Bot

Source: [Wikidata:Creating a bot](https://www.wikidata.org/wiki/Wikidata:Creating_a_bot)

There are three Python based bot libraries of note:
- [Pywikibot](https://github.com/wikimedia/pywikibot)
  - See: [Wikidata:Creating_a_bot#Pywikibot](https://www.wikidata.org/wiki/Wikidata:Creating_a_bot#Pywikibot)
  - For interacting with the MediaWiki API, so not just Wikidata
  - **Note**: at time of writing has incomplete support for lexemes (language data)
- [Wikidata Integrator](https://github.com/SuLab/WikidataIntegrator)
  - See: [Wikidata:Creating_a_bot#Wikidata_Integrator](https://www.wikidata.org/wiki/Wikidata:Creating_a_bot#Wikidata_Integrator)
  - Has high integration with the Wikidata SPARQL endpoint
- [Wikibase Integrator](https://github.com/LeMyst/WikibaseIntegrator)
  - See: [Wikidata:Creating_a_bot#WikibaseIntegrator](https://www.wikidata.org/wiki/Wikidata:Creating_a_bot#WikibaseIntegrator)
  - Even easier to create bots, especially for [Wikibase](https://wikiba.se/) instances
  - Seems to have more support, and the [usage examples](https://github.com/LeMyst/WikibaseIntegrator/tree/master/notebooks)/[docs](https://wikibaseintegrator.readthedocs.io/en/stable/) are more expansive
  - Most of what you need is in the [readme](https://github.com/LeMyst/WikibaseIntegrator?tab=readme-ov-file#wikibase-integrator)
  
We'll include a basic example of interacting with the [Wikidata item sandbox](https://www.wikidata.org/wiki/Q4115189) for Wikibase Integrator.

## Imports

In [1]:
# pip install wikibaseintegrator

In [2]:
%load_ext jupyter_black

In [3]:
from wikibaseintegrator import WikibaseIntegrator, datatypes, wbi_login
from wikibaseintegrator.datatypes import Item
from wikibaseintegrator.models import Qualifiers, References, Reference
from wikibaseintegrator.wbi_config import config

## Wikibase Integrator Bot Examples

For this example, we'll be using [Andrew McAllister Bot (WMDE)](https://www.wikidata.org/wiki/User:Andrew_McAllister_Bot_(WMDE)).

### Login and Create Object

In [4]:
# Make a user agent string that clearly states the bot account.
USER_AGENT_STRING = "Andrew McAllister Bot (WMDE)/1.0 (https://www.wikidata.org/wiki/User:Andrew McAllister Bot (WMDE))"

In [5]:
config["USER_AGENT"] = USER_AGENT_STRING

In [6]:
wd_user = "Andrew McAllister Bot (WMDE)"
wd_pass = "MY_BOT_ACCOUNT_PASS"
login_wikidata = wbi_login.Login(
    user=wd_user,
    password=wd_pass,
    mediawiki_api_url="https://www.wikidata.org/w/api.php",
)

In [7]:
# Add login credentials to your WikibaseIntegrator object so they're applied to all requests.
wbi = WikibaseIntegrator(login=login_wikidata)

### Access Basic Item Information

In [8]:
WD_ITEM_SANDBOX_QID = "Q4115189"

In [9]:
WD_ACTION_API_ENDPOINT = "https://www.wikidata.org/w/api.php"

In [10]:
sandbox_entity = wbi.item.get(
    WD_ITEM_SANDBOX_QID,
    mediawiki_api_url=WD_ACTION_API_ENDPOINT,
)

In [11]:
sandbox_entity.labels.get("en").value

'Wikidata Sandbox'

In [12]:
sandbox_entity.descriptions.get("en").value

'This is a sandbox for testing changes to items. Please be gentle with it. Feel free to change anything on this page! For testing links, try adding ones to userpages.'

In [13]:
sandbox_entity_instance_of_qid = sandbox_entity.claims.get("P31")[0].mainsnak.datavalue[
    "value"
]["id"]
sandbox_entity_instance_of_qid

'Q21281405'

In [14]:
sandbox_entity_instance_of_qid_entity = wbi.item.get(
    sandbox_entity_instance_of_qid,
    mediawiki_api_url=WD_ACTION_API_ENDPOINT,
)

In [15]:
sandbox_entity_instance_of_qid_entity.labels.get("en").value

'Wikidata internal entity'

### Editing the Item

In [16]:
# Add the "maintained by" (P126) claim referencing the 'Tester' family name (Q37528637).
sandbox_entity.claims.add(Item(prop_nr="P126", value="Q37528637"))

<Claims @f10910 _Claims__claims={'P460': [<Item @13c450 _Claim__mainsnak=<Snak @607990 _Snak__snaktype=<WikibaseSnakType.KNOWN_VALUE: 'value'> _Snak__property_number='P460' _Snak__hash='b1faa2726e622bc3727462634d5fb3084e7756b2' _Snak__datavalue={'value': {'entity-type': 'item', 'numeric-id': 13406268, 'id': 'Q13406268'}, 'type': 'wikibase-entityid'} _Snak__datatype='wikibase-item'> _Claim__type='statement' _Claim__qualifiers=<Qualifiers @5fed50 _Qualifiers__qualifiers={}> _Claim__qualifiers_order=[] _Claim__id='Q4115189$0ce92f0e-46e8-fb05-bd41-e19f983485d0' _Claim__rank=<WikibaseRank.NORMAL: 'normal'> _Claim__removed=False _Claim__references=<References @9fa50 _References__references=[]>>, <Item @605110 _Claim__mainsnak=<Snak @f117d0 _Snak__snaktype=<WikibaseSnakType.KNOWN_VALUE: 'value'> _Snak__property_number='P460' _Snak__hash='d58d2897e1b93269cfa377b0b972022d2a623cc0' _Snak__datavalue={'value': {'entity-type': 'item', 'numeric-id': 15397819, 'id': 'Q15397819'}, 'type': 'wikibase-en

In [17]:
sandbox_entity.write()

<ItemEntity @78ec50 _BaseEntity__api=<wikibaseintegrator.wikibaseintegrator.WikibaseIntegrator object at 0x717ea052a190>
	 _BaseEntity__title='Q4115189'
	 _BaseEntity__pageid=4246474
	 _BaseEntity__lastrevid=2117379539
	 _BaseEntity__type='item'
	 _BaseEntity__id='Q4115189'
	 _BaseEntity__claims=<Claims @4efb90 _Claims__claims={'P460': [<Item @101890 _Claim__mainsnak=<Snak @537950 _Snak__snaktype=<WikibaseSnakType.KNOWN_VALUE: 'value'> _Snak__property_number='P460' _Snak__hash='b1faa2726e622bc3727462634d5fb3084e7756b2' _Snak__datavalue={'value': {'entity-type': 'item', 'numeric-id': 13406268, 'id': 'Q13406268'}, 'type': 'wikibase-entityid'} _Snak__datatype='wikibase-item'> _Claim__type='statement' _Claim__qualifiers=<Qualifiers @534b90 _Qualifiers__qualifiers={}> _Claim__qualifiers_order=[] _Claim__id='Q4115189$0ce92f0e-46e8-fb05-bd41-e19f983485d0' _Claim__rank=<WikibaseRank.NORMAL: 'normal'> _Claim__removed=False _Claim__references=<References @5358d0 _References__references=[]>>, <It

## Creating a New Item

Feel free to give try out the [item creation example](https://github.com/LeMyst/WikibaseIntegrator/blob/master/notebooks/item_create_new.ipynb) on [Test Wikidata](https://test.wikidata.org/wiki/Wikidata:Main_Page)!